In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression as LR

In [2]:
tweet_df_location = pd.read_csv('../data/graphAnalysis/clean_climateTwitterData_Jan20.csv')

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
X = tweet_df_location['text']
y = tweet_df_location['search_hashtags']


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
text_Vect = TfidfVectorizer()
X_Vect  = text_Vect.fit_transform(X)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_Vect, y, test_size=0.30, random_state=42, stratify=y)

In [7]:
X_train.shape

(50683, 83820)

In [8]:
y_train.shape

(50683,)

In [9]:
X_test.shape

(21722, 83820)

In [10]:
y_test.shape

(21722,)

In [11]:
lr = LR()

In [12]:
lr.fit(X_train, y_train)  

c:\users\tasne\envs\newproject\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
predictions = lr.predict(X_test)

In [14]:
from sklearn.metrics import confusion_matrix, classification_report

In [15]:
print(confusion_matrix(y_test,predictions))

[[ 414    1   16   43   28   34    5    3    1   14    0    9]
 [   0  226    0   33    8    0    2    0    1    0    0    0]
 [   9    0 1256  228   57  249   14   24   25   24    4   23]
 [  22    6  201 3959  115  250   52   17  123   50   14   48]
 [   9    7   61  179 1027  120   11    7   26   38    4    6]
 [   6    4   65  144   32 5142   17   45   23   16    2   11]
 [   5    0   13   87    7   11 1182    0   25    3   10   68]
 [   2    4   31   34    8  329    2  485    5    4    4    3]
 [   0    0    7   87   20   31   19    2 1075    4    0    1]
 [  11    0   10   59   27   44    1    2    6 1207    3    7]
 [   0    0    3    9    5    6   23    1   16    2  353   12]
 [   2    0    9   49    7    9   51    0   22    3    9 1576]]


In [16]:
print(classification_report(y_test,predictions))

                   precision    recall  f1-score   support

    #actonclimate       0.86      0.73      0.79       568
       #bushfires       0.91      0.84      0.87       270
   #climateaction       0.75      0.66      0.70      1913
   #climatechange       0.81      0.82      0.81      4857
   #climatecrisis       0.77      0.69      0.72      1495
   #climatestrike       0.83      0.93      0.88      5507
     #environment       0.86      0.84      0.85      1411
#fridaysforfuture       0.83      0.53      0.65       911
   #globalwarming       0.80      0.86      0.83      1246
    #greennewdeal       0.88      0.88      0.88      1377
   #savetheplanet       0.88      0.82      0.85       430
  #sustainability       0.89      0.91      0.90      1737

         accuracy                           0.82     21722
        macro avg       0.84      0.79      0.81     21722
     weighted avg       0.82      0.82      0.82     21722



In [17]:
from sklearn import metrics

In [18]:
metrics.accuracy_score(y_test,predictions)

0.8241414234416721

Accuracy is --- 0.8241414234416721

In [21]:
strVal = ["Schools strikes in sweden"]
Value = text_Vect.transform(strVal)
Value.shape

(1, 83820)

In [22]:
lr.predict(Value)

array(['#climatestrike'], dtype=object)